# Using PuLP to Optimize Lineups

## Integer Programming for Optimization

NEED TO EDIT THIS

Linear Programing
The simplest type of mathematical program is a linear program. For your mathematical program to be a linear program you need the following conditions to be true:

The decision variables must be real variables;

The objective must be a linear expression;

The constraints must be linear expressions.

Linear expressions are any expression of the form

a1x1+a2x2+a3x3+...anxn{<=,=,>=}b
where the ai and b are known constants and xi are variables. The process of solving a linear program is called linear programing. Linear programing is done via the Revised Simplex Method (also known as the Primal Simplex Method), the Dual Simplex Method or an Interior Point Method. Some solvers like cplex allow you to specify which method you use, but we won’t go into further detail here.

Integer Programing
Integer programs are almost identical to linear programs with one very important exception. Some of the decision variables in integer programs may need to have only integer values. The variables are known as integer variables. Since most integer programs contain a mix of continuous variables and integer variables they are often known as mixed integer programs. While the change from linear programing is a minor one, the effect on the solution process is enormous. Integer programs can be very difficult problems to solve and there is a lot of current research finding “good” ways to solve integer programs. Integer programs can be solved using the branch-and-bound process.

Note For MIPs of any reasonable size the solution time grows exponentially as the number of integer variables increases.

## Loading Player Data

In [ ]:
from pprint import pprint

import pandas as pd
import pulp

In [2]:
df = pd.read_csv('data/pool.csv')

In [3]:
df.head()

,first_name,last_name,pos,team,salary,fppg,full_name,opp
0,Buffalo,Defense,DST,BUF,3200,21.0,Buffalo_Defense,MIA
1,LA Rams,Defense,DST,LAR,3300,20.0,LA Rams_Defense,ARI
2,New Orleans,Defense,DST,NOR,3400,17.0,New Orleans_Defense,CAR
3,Baltimore,Defense,DST,BAL,4200,11.0,Baltimore_Defense,CIN
4,Denver,Defense,DST,DEN,2400,10.0,Denver_Defense,LVR


## Setting up the PuLP problem

As noted in the excellent [PuLP documentation](https://coin-or.github.io/pulp/main/the_optimisation_process.html), the process of solving an optimization problem can be broken down into five general steps:

* Getting the problem description

* Formulating the mathematical program

* Solving the mathematical program

* Performing some post-optimal analysis

* Presenting the solution and analysis

### Getting the problem description

In plain language, we want to maximize the fantasy points of a valid lineup. 

Fantasy points can be projected (upcoming week/season) or actual (historical research). 

A valid lineup complies with the site's (examples here are DraftKings NFL):
    
* Position rules: e.g. 1 QB, 2 RB, 3 WR, 1 TE, 1 DST, 1 FLEX
* Team/Game rules: no more than n players from one team or game
* Salary cap: e.g. lineup cannot cost more than $50K (DK)

### Formulating the mathematical program

There is a mathematical notation for these type of programs that looks like

![IP Canonical Form](https://wikimedia.org/api/rest_v1/media/math/render/svg/639c4281a57140db9a4416ca58f9d9af14243bb0)

I don't have any formal math training, so this notation is not very natural for me, but it does help you frame a problem in a way that it can be translated into code. As we will see below, it can be challenging to express a concept (stacking a QB with at least one player from the same team) as a linear constraint (sum of teammates - 1 * QB >= 0).

We'll use the PuLP library to setup the problem, create the objective function, and add constraints. In the end, the problem formulation in python code will represent the same thing as the mathematical notation.

#### Problem Setup

Setting up a problem in PuLP is straightforward - you just need a name and a problem type. Here, the problem name is ```FPOpt``` but you can choose any name you like - it doesn't really matter. We want to maximize fantasy points, so the problem type is ```pulp.LpMaximize```.

In [4]:
# setup problem
prob = pulp.LpProblem('FPOpt', pulp.LpMaximize)

If we inspect the ```prob``` object, we see that it is a maximization problem with no objective function and no variables. Not very useful at this point, but this is only the first step.

In [5]:
prob

FPOpt:
MAXIMIZE
None
VARIABLES

Now we need to create the problem variables. Here, we are going to create a set of binary variables (value can either be 0 or 1) representing the player's absence (0) or presence (1) in the optimal lineup. The algorithm will then discover the optimal combination of 0 and 1 values to maximize fantasy points and satisfy all the constraints. This is an Integer Programming problem, because you cannot put a fractional player in your lineup -- the value is either 0 or 1 for each player variable.

The simplest way to store the variables is as a dictionary, where the key is a string that is associated with one variable (here, a player's id or full_name), and the value is the corresponding binary LpVariable. You could store the variables in a different data structure, such as a list or tuple, but the dictionary offers quick and easy lookup by a string, rather than having to keep track of index number to access the right element in a list.

You can create a dictionary manually, but PuLP has a convenience function ```pulp.LpVariable.dicts``` that makes it very easy to create a dictionary from a python iterable. The function parameters are the variable name prefix, the iterable, and the variable type.

In the example below, the ```player_vars``` dict is created by iterating over the ```full_name``` column in the dataframe and creating a binary variable named 'Player_{full_name}' for each row in the column.

In [9]:
# create variables
# key is Player_{full_name}, value is pulp.LpVariable
player_vars = pulp.LpVariable.dicts('Player', df.full_name, cat='Binary')

Let's take a look at an individual item in the dictionary. We can get the first key by casting the keys to a list and subscripting element 0. We can then look up the value of the first item with that key.

In the example below, the first dictionary item represents the Buffalo Bills DST. The key is a string -- Buffalo_Defense -- which is the player name. The value is a pulp.pulp.LpVariable with the name Player_Buffalo_Defense. 

The LpVariable's __repr__ method only returns the object's name (here, Player_Buffalo_Defense) when we display the object. If we want to see all of its associated values, we can pretty print it as a dictionary using ```pprint``` and ```vars```. When we do this, we see that a Binary variable is not a distinct category. Instead, it is shorthand for an Integer variable with a lowBound of 0 and and upBound of 1.

In [22]:
# we only get the name if we call print({LpVariable instance})
pulp.pulp.LpVariable.__repr__??

Signature: pulp.pulp.LpVariable.__repr__(self)
Docstring: Return repr(self).
Source:   
    def __repr__(self):
        return self.name
File:      c:\users\erict\miniconda3\envs\ffenv\lib\site-packages\pulp\pulp.py
Type:      function


In [23]:
# take a look at dict item
# key is str, value is LpVariable
k = list(player_vars.keys())[0]
print(k, type(k))
print(player_vars[k], type(player_vars[k]))
pprint(vars(player_vars[k]))

Buffalo_Defense <class 'str'>
Player_Buffalo_Defense <class 'pulp.pulp.LpVariable'>
{'_LpElement__name': 'Player_Buffalo_Defense',
 '_lowbound_original': None,
 '_upbound_original': None,
 'cat': 'Integer',
 'dj': None,
 'hash': 1911320924680,
 'lowBound': 0,
 'modified': True,
 'upBound': 1,
 'varValue': None}


To sum it up, we need to create a binary variable for each player. If the variable is set to 1 in the optimal solution, the player is in the lineup. If the variable is set to 0, the player is not in the lineup.

The python dictionary is a convenient way of storing these variables, and pulp makes it even easier with the ```LpVariable.dicts``` convenience method.

This is an integer program because we cannot have .25 of a player in a lineup. Keep in mind that the time to solve an integer program increases exponentially as the number of variables increase. It is thus advisable to remove clearly inferior players from the initial pool (I use positional points thresholds to weed them out).

## Adding the Objective Function

In python 3.7+, a dictionary preserves the order of the keys as items are added, which allows for parallel iteration. The commented out code shows how this can be implemented in earlier versions of the language, but I will assume from here on out that you are using at least python 3.7.

In [26]:
# objective function - sum of fantasy points
prob += pulp.lpSum([pts * pvar for pvar, pts in zip(player_vars.values(), df.fppg)])

# objective function for versions < python3.7
# step one: create a dictionary where the keys are full_name and values are fppg
# fppg_dict = dict(zip(df.full_name, df.fppg))
#
# step two: iterate over the keys in player_vars(which are identical to those in fppg_dict)
#           and use those keys to get the corresponding LpVariable and fppg value
# prob += pulp.lpSum([fppg_dict[player_name] * player_vars[player_name] for player_name in player_vars])

When we look at ```prob``` again, we have much more information.

MAXIMIZE is the same. This is still a maximization problem.

There is a long equation that is the objective function. Remember that the variables are binary -- set to either 0 or 1. So if, for example, AJ Brown is in our lineup, that part of the equation evaluates to 34.1 * 1, or 34.1. Players that are not in the lineup will evaluate to zero because any number multiplied by zero is zero.

We also see a list of variables that are named after the players that must be an integer and either 0 or 1.

In [32]:
prob

FPOpt:
MAXIMIZE
34.1*Player_A.J._Brown + 18.5*Player_Aaron_Jones + 26.0*Player_Aaron_Rodgers + 9.7*Player_Adam_Thielen + 12.3*Player_Adrian_Peterson + 29.5*Player_Alexander_Mattison + 5.7*Player_Allen_Robinson + 10.1*Player_Amari_Cooper + 9.7*Player_Ameer_Abdullah + 13.52*Player_Andy_Dalton + 39.8*Player_Antonio_Brown + 10.0*Player_Arizona_Defense + 8.1*Player_Austin_Carr + 17.1*Player_Austin_Ekeler + 13.7*Player_Austin_Hooper + 16.24*Player_Baker_Mayfield + 11.0*Player_Baltimore_Defense + 42.6*Player_Brandin_Cooks + 6.6*Player_Braxton_Berrios + 11.4*Player_Breshad_Perriman + 16.6*Player_Brian_Hill + 13.1*Player_Bryan_Edwards + 21.0*Player_Buffalo_Defense + 15.2*Player_Byron_Pringle + 14.92*Player_C.J._Beathard + 6.4*Player_C.J._Ham + 11.6*Player_Calvin_Ridley + 11.6*Player_Cam_Akers + 38.48*Player_Cam_Newton + 9.6*Player_CeeDee_Lamb + 12.4*Player_Chad_Beebe + 16.72*Player_Chad_Henne + 24.1*Player_Chase_Claypool + 10.3*Player_Chris_Carson + 15.7*Player_Chris_Conley + 33.3*Player_Chris_

## Core Constraints

In [33]:
# constraint: salary must not exceed cap
prob += pulp.lpSum([sal * pvar for pvar, sal in zip(player_vars.values(), df.salary)]) <= 50000

Let's take a look at the problem again. We now see SUBJECT TO _C1, which is a linear equation for the salary cap. The sum of the chosen player variables (1) * salary must be <= 50000.

In [36]:
prob

FPOpt:
MAXIMIZE
34.1*Player_A.J._Brown + 18.5*Player_Aaron_Jones + 26.0*Player_Aaron_Rodgers + 9.7*Player_Adam_Thielen + 12.3*Player_Adrian_Peterson + 29.5*Player_Alexander_Mattison + 5.7*Player_Allen_Robinson + 10.1*Player_Amari_Cooper + 9.7*Player_Ameer_Abdullah + 13.52*Player_Andy_Dalton + 39.8*Player_Antonio_Brown + 10.0*Player_Arizona_Defense + 8.1*Player_Austin_Carr + 17.1*Player_Austin_Ekeler + 13.7*Player_Austin_Hooper + 16.24*Player_Baker_Mayfield + 11.0*Player_Baltimore_Defense + 42.6*Player_Brandin_Cooks + 6.6*Player_Braxton_Berrios + 11.4*Player_Breshad_Perriman + 16.6*Player_Brian_Hill + 13.1*Player_Bryan_Edwards + 21.0*Player_Buffalo_Defense + 15.2*Player_Byron_Pringle + 14.92*Player_C.J._Beathard + 6.4*Player_C.J._Ham + 11.6*Player_Calvin_Ridley + 11.6*Player_Cam_Akers + 38.48*Player_Cam_Newton + 9.6*Player_CeeDee_Lamb + 12.4*Player_Chad_Beebe + 16.72*Player_Chad_Henne + 24.1*Player_Chase_Claypool + 10.3*Player_Chris_Carson + 15.7*Player_Chris_Conley + 33.3*Player_Chris_

In [38]:
# constraint: lineup must have 9 players
prob += pulp.lpSum(player_vars.values()) == 9

Let's take a look at the problem again. We now see SUBJECT TO _C2, which is a linear equation for the number of players. The sum of the chosen player variables (1) must be = 9.

In [39]:
prob

ique_Dafney + 12.7*Player_Donald_Parham + 20.04*Player_Drew_Brees + 27.26*Player_Drew_Lock + 5.3*Player_Durham_Smythe + 21.3*Player_Emmanuel_Sanders + 15.1*Player_Ezekiel_Elliott + 21.7*Player_Gabriel_Davis + 13.8*Player_George_Kittle + 6.0*Player_Green_Bay_Defense + 8.0*Player_Gus_Edwards + 12.8*Player_Hayden_Hurst + 5.8*Player_Henry_Ruggs_III + 5.4*Player_Hunter_Renfrow + 9.0*Player_Indianapolis_Defense + 6.7*Player_Isaiah_Ford + 30.5*Player_Isaiah_McKenzie + 31.0*Player_J.K._Dobbins + 18.26*Player_Jakobi_Meyers + 11.2*Player_James_Conner + 13.1*Player_James_White + 7.1*Player_Jamison_Crowder + 14.3*Player_Jared_Cook + 16.4*Player_Jarvis_Landry + 23.8*Player_Jeff_Wilson_Jr. + 30.0*Player_Jerry_Jeudy + 17.2*Player_John_Brown + 13.84*Player_John_Wolford + 41.4*Player_Jonathan_Taylor + 8.1*Player_Jonathan_Ward + 13.9*Player_Josh_Adams + 20.26*Player_Josh_Allen + 20.9*Player_Josh_Jacobs + 6.9*Player_Josh_Reynolds + 18.5*Player_JuJu_Smith_Schuster + 35.98*Player_Justin_Herbert + 13.4*Play

Now we are going to add the positional constraints. You can accomodate the FLEX by setting minimum and maximum counts for the flex-eligible positions. So, here, the constraint is that a lineup must have 2 or more RBs and 3 or less RBs, which meets the minimum requirement of 2 and also allows a lineup to use a third RB in the flex. The same logic applies to the constraints for WR and TE.

In [40]:
# add positional constraints
prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                    if pos == 'QB']) == 1
prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                    if pos == 'RB']) >= 2
prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                    if pos == 'RB']) <= 3
prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                    if pos == 'WR']) >= 3
prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                    if pos == 'WR']) <= 4
prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                    if pos == 'TE']) >= 1
prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                    if pos == 'TE']) <= 2
prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                    if pos == 'DST']) == 1

Let's take a look at prob again. We can see that the problem is growing in complexity as we add more constraints. This latest round takes us all the way up to _C10.

In [41]:
prob

ilson
 + 7000 Player_Ryan_Tannehill + 4400 Player_Salvon_Ahmed
 + 4900 Player_Sam_Darnold + 4700 Player_Samaje_Perine
 + 4600 Player_Sony_Michel + 8100 Player_Stefon_Diggs
 + 2600 Player_Stephen_Anderson + 5200 Player_Sterling_Shepard
 + 4600 Player_T.J._Hockenson + 5800 Player_T.Y._Hilton
 + 5800 Player_Taysom_Hill + 5300 Player_Teddy_Bridgewater
 + 4000 Player_Theo_Riddick + 4000 Player_Tim_Patrick + 7200 Player_Tom_Brady
 + 5900 Player_Tony_Pollard + 4000 Player_Trayveon_Williams
 + 3000 Player_Trent_Sherfield + 2500 Player_Troy_Fumagalli
 + 5100 Player_Tua_Tagovailoa + 4300 Player_Ty_Johnson
 + 4000 Player_Ty_Montgomery + 3000 Player_Tyler_Conklin
 + 3500 Player_Tyler_Higbee + 5800 Player_Tyler_Lockett
 + 3000 Player_Tyrie_Cleveland + 4000 Player_Tyron_Johnson
 + 3000 Player_Van_Jefferson + 2500 Player_Vance_McDonald
 + 5300 Player_Wayne_Gallman + 4200 Player_Zach_Pascal <= 50000

_C2: Player_A.J._Brown + Player_Aaron_Jones + Player_Aaron_Rodgers
 + Player_Adam_Thielen + Player_Adr

## Optional Constraints

It can be desirable for a lineup to include a QB and a WR from the same team  

In [26]:
# stacking QB and receiver
for row in df.loc[df.pos == 'QB', :].itertuples(index=False):
    prob += pulp.lpSum([v for k,v in player_vars.items() if
        df.loc[df.full_name == k, 'team'].values[0] == row.team and
                  df.loc[df.full_name == k, 'pos'].values[0] == 'WR'] +
                  [-1 * player_vars[row.full_name]]) >= 0  

In [11]:
# stacking QB and two other players
for row in df.loc[df.pos == 'QB', :].itertuples(index=False):
    prob += pulp.lpSum([v for k,v in player_vars.items() if
                        df.loc[df.full_name == k, 'team'].values[0] == row.team and
                        df.loc[df.full_name == k, 'pos'].values[0] in ('WR', 'TE', 'RB')] +
                        [-2 * player_vars[row.full_name]]) >= 0  

In [ ]:
# include a bringback
for row in df.loc[df.pos == 'QB', :].itertuples(index=False):
    prob += pulp.lpSum([v for k,v in player_vars.items() if
        df.loc[df.full_name == k, 'team'].values[0] == row.opp and
                  df.loc[df.full_name == k, 'pos'].values[0] == 'WR'] +
                  [-1 * player_vars[row.full_name]]) >= 0

## Solve Problem

In [13]:
# solve problem and print solution
prob.solve()
chosen = [v.name for v in prob.variables() if v.varValue == 1]
solution = df.loc[df.full_name.isin([c.replace('Player_', '') for c in chosen]), list(df.columns)[0:-1]]
print(solution)
print(f'\n{solution.fppg.sum()}')

    first_name last_name  pos team  salary   fppg
0      Buffalo   Defense  DST  BUF    3200  21.00
14         Tom     Brady   QB  TAM    7200  34.26
42    Jonathan    Taylor   RB  IND    7400  41.40
45      Darwin  Thompson   RB  KAN    4400  30.00
87       Chris   Herndon   TE  NYJ    2800  19.30
110    Brandin     Cooks   WR  HOU    6900  42.60
111     Marvin     Jones   WR  DET    5100  41.00
112    Antonio     Brown   WR  TAM    5500  39.80
115      Chris    Godwin   WR  TAM    6600  33.30

302.66
